# Applied Data Science Capstone Project
___

The objective of this notebook is to perform a capstone project with the location data provided by Fourquare API.

## Primary Import of Data Science Libraries

In [2]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')